In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd

In [0]:
import os

PATH_TO_DATA = '/content/gdrive/My Drive/mlcourse/dota/'

for dirname, _, filenames in os.walk(PATH_TO_DATA):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**1. Download data**

In [0]:
df_train_features = pd.read_csv(PATH_TO_DATA + 'df_train_features.csv', index_col = 'match_id_hash')

In [0]:
df_train_targets = pd.read_csv(PATH_TO_DATA + 'df_train_targets.csv', index_col='match_id_hash')

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=17)

In [0]:
y = df_train_targets['radiant_win'].values

In [0]:
from sklearn.model_selection import ShuffleSplit, KFold
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=17)

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
heroes_col = ['r1_hero_id',
              'r2_hero_id',
              'r3_hero_id',
              'r4_hero_id',
              'r5_hero_id',
              'd1_hero_id',
              'd2_hero_id',
              'd3_hero_id',
              'd4_hero_id',
              'd5_hero_id',
              ]

In [0]:
df_train_heroes = df_train_features[heroes_col]

In [0]:
df_train_heroes.head()

,r1_hero_id,r2_hero_id,r3_hero_id,r4_hero_id,r5_hero_id,d1_hero_id,d2_hero_id,d3_hero_id,d4_hero_id,d5_hero_id
match_id_hash,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,11,78,14,59,77,12,21,60,84,34
b9c57c450ce74a2af79c9ce96fac144d,15,96,27,63,89,58,14,1,56,92
6db558535151ea18ca70a6892197db41,101,51,44,49,53,18,67,47,40,17
46a0ddce8f7ed2a8d9bd5edcbb925682,14,99,101,26,41,18,98,8,69,86
b1b35ff97723d9b7ade1c9c3cf48f770,42,69,27,104,65,23,22,35,72,1


In [0]:
df_heroes_r1 = pd.get_dummies(df_train_heroes['r1_hero_id'], prefix = 'r')
df_heroes_r2 = pd.get_dummies(df_train_heroes['r2_hero_id'], prefix = 'r')
df_heroes_r3 = pd.get_dummies(df_train_heroes['r3_hero_id'], prefix = 'r')
df_heroes_r4 = pd.get_dummies(df_train_heroes['r4_hero_id'], prefix = 'r')
df_heroes_r5 = pd.get_dummies(df_train_heroes['r5_hero_id'], prefix = 'r')

In [0]:
df_heroes_r = df_heroes_r1 | df_heroes_r2 | df_heroes_r3 | df_heroes_r4 | df_heroes_r5

In [0]:
df_heroes_d1 = pd.get_dummies(df_train_heroes['d1_hero_id'], prefix = 'd')
df_heroes_d2 = pd.get_dummies(df_train_heroes['d2_hero_id'], prefix = 'd')
df_heroes_d3 = pd.get_dummies(df_train_heroes['d3_hero_id'], prefix = 'd')
df_heroes_d4 = pd.get_dummies(df_train_heroes['d4_hero_id'], prefix = 'd')
df_heroes_d5 = pd.get_dummies(df_train_heroes['d5_hero_id'], prefix = 'd')
df_heroes_d = df_heroes_d1 | df_heroes_d2 | df_heroes_d3 | df_heroes_d4 | df_heroes_d5

In [0]:
df_heroes_d.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,...,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_119,d_120
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6db558535151ea18ca70a6892197db41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
b1b35ff97723d9b7ade1c9c3cf48f770,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_heroes_ohe = pd.concat([df_heroes_r, df_heroes_d], axis = 1)

In [0]:
X = df_heroes_ohe.values
cv_score = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print(round(cv_score.mean(), 5))

0.60566


In [0]:
df_train_features = pd.read_csv(PATH_TO_DATA + 'df_train_features_ext.csv', index_col = 'match_id_hash')

In [0]:
X = scaler.fit_transform(pd.concat([df_train_features[['r_total_gold', 'd_total_gold']], df_heroes_ohe], axis = 1))
cv_score = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print(round(cv_score.mean(), 5))

0.82175


In [0]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier()
X = df_heroes_ohe.values
cv_score = cross_val_score(model_xgb, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
print(round(cv_score.mean(), 5))

0.58817


In [0]:
import lightgbm
from sklearn.model_selection import train_test_split
X = df_heroes_ohe.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17, stratify=y)
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data  = lightgbm.Dataset(X_test, label=y_test)

In [0]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model_lgbm = lightgbm.train(parameters,
                            train_data,
                            valid_sets=test_data,
                            num_boost_round=5000,
                            early_stopping_rounds=100)

[1]	valid_0's auc: 0.543706
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.553074
[3]	valid_0's auc: 0.557193
[4]	valid_0's auc: 0.56205
[5]	valid_0's auc: 0.560739
[6]	valid_0's auc: 0.56441
[7]	valid_0's auc: 0.565112
[8]	valid_0's auc: 0.564386
[9]	valid_0's auc: 0.563694
[10]	valid_0's auc: 0.563992
[11]	valid_0's auc: 0.565381
[12]	valid_0's auc: 0.565393
[13]	valid_0's auc: 0.564021
[14]	valid_0's auc: 0.565116
[15]	valid_0's auc: 0.566744
[16]	valid_0's auc: 0.567447
[17]	valid_0's auc: 0.567375
[18]	valid_0's auc: 0.567804
[19]	valid_0's auc: 0.568485
[20]	valid_0's auc: 0.566968
[21]	valid_0's auc: 0.568361
[22]	valid_0's auc: 0.569632
[23]	valid_0's auc: 0.570248
[24]	valid_0's auc: 0.571024
[25]	valid_0's auc: 0.571804
[26]	valid_0's auc: 0.572126
[27]	valid_0's auc: 0.572942
[28]	valid_0's auc: 0.573129
[29]	valid_0's auc: 0.573991
[30]	valid_0's auc: 0.574216
[31]	valid_0's auc: 0.57417
[32]	valid_0's auc: 0.574719
[33]	valid_0's auc: 0